## Multiple Agents Concurrently

In [118]:
import asyncio
from dataclasses import dataclass

from autogen_core.application import SingleThreadedAgentRuntime
from autogen_core.base import AgentId, MessageContext, TopicId
from autogen_core.components import (
    DefaultTopicId,
    RoutedAgent,
    default_subscription,
    message_handler,
    type_subscription,
)


@dataclass
class Task:
    task_id: str


@dataclass
class TaskResponse:
    task_id: str
    result: str

## Single Message & Multiple Processors
The first pattern shows how a single message can be processed by multiple agents simultaneously:

- Each `Processor` agent subscribes to the default topic using the {py:meth}`~autogen_core.components.default_subscription` decorator
- When publishing a message to the default topic, all registered agents will process the message independently

In [129]:
@default_subscription
class Processor(RoutedAgent):
    @message_handler
    async def on_task(self, message: Task, ctx: MessageContext) -> None:
        print(f"{self._description} starting task {message.task_id}")
        await asyncio.sleep(2)  # Simulate work
        print(f"{self._description} finished task {message.task_id}")

In [130]:
runtime = SingleThreadedAgentRuntime()

await Processor.register(runtime, "agent_1", lambda: Processor("Agent 1"))
await Processor.register(runtime, "agent_2", lambda: Processor("Agent 2"))

runtime.start()

await runtime.publish_message(Task(task_id="task-1"), topic_id=DefaultTopicId())

await runtime.stop_when_idle()

Agent 1 starting task task-1
Agent 2 starting task task-1
Agent 1 finished task task-1
Agent 2 finished task task-1


## Multiple messages & Multiple Processors
Second, this pattern demonstrates routing different types of messages to specific processors:
- `UrgentProcessor` subscribes to the "urgent" topic
- `NormalProcessor` subscribes to the "normal" topic

We make an agent subscribe to a specific topic type using the {py:meth}`~autogen_core.components.type_subscription` decorator.

In [122]:
@type_subscription(topic_type="urgent")
class UrgentProcessor(RoutedAgent):
    @message_handler
    async def on_task(self, message: Task, ctx: MessageContext) -> None:
        print(f"Urgent processor starting task {message.task_id}")
        await asyncio.sleep(1)  # Simulate work
        print(f"Urgent processor finished task {message.task_id}")


@type_subscription(topic_type="normal")
class NormalProcessor(RoutedAgent):
    @message_handler
    async def on_task(self, message: Task, ctx: MessageContext) -> None:
        print(f"Normal processor starting task {message.task_id}")
        await asyncio.sleep(3)  # Simulate work
        print(f"Normal processor finished task {message.task_id}")

After registering the agents, we can publish messages to the "urgent" and "normal" topics:

In [117]:
runtime = SingleThreadedAgentRuntime()

await UrgentProcessor.register(runtime, "urgent_processor", lambda: UrgentProcessor("Urgent Processor"))
await NormalProcessor.register(runtime, "normal_processor", lambda: NormalProcessor("Normal Processor"))

runtime.start()

await asyncio.gather(
    runtime.publish_message(Task(task_id="normal-1"), topic_id=TopicId(type="normal", source="default")),
    runtime.publish_message(Task(task_id="urgent-1"), topic_id=TopicId(type="urgent", source="default")),
)

await runtime.stop_when_idle()

Normal processor starting task normal-1
Urgent processor starting task urgent-1
Urgent processor finished task urgent-1
Normal processor finished task normal-1


## Direct Messages

Finally, in contrast to the previous patterns, this pattern is about direct messages. It demonstrates two ways to send messages:

- Direct messaging between agents
- Sending messages from the runtime to specific agents

Things to consider when using direct messaging:
- Messages are addressed using {py:class}`~autogen_core.components.AgentId`
- The sender can expect and receive a response from the target agent

In [126]:
class WorkerAgent(RoutedAgent):
    @message_handler
    async def on_task(self, message: Task, ctx: MessageContext) -> TaskResponse:
        print(f"{self.id} starting task {message.task_id}")
        await asyncio.sleep(2)  # Simulate work
        print(f"{self.id} finished task {message.task_id}")
        return TaskResponse(task_id=message.task_id, result=f"Results by {self.id}")


@default_subscription
class DelegatorAgent(RoutedAgent):
    def __init__(self, description: str, worker_type: str):
        super().__init__(description)
        self.worker_instances = [AgentId(worker_type, f"{worker_type}-1"), AgentId(worker_type, f"{worker_type}-2")]

    @message_handler
    async def on_task(self, message: Task, ctx: MessageContext) -> TaskResponse:
        print(f"Delegator received task {message.task_id}.")

        subtask1 = Task(task_id="task-part-1")
        subtask2 = Task(task_id="task-part-2")

        worker1_result, worker2_result = await asyncio.gather(
            self.send_message(subtask1, self.worker_instances[0]), self.send_message(subtask2, self.worker_instances[1])
        )

        combined_result = f"Part 1: {worker1_result.result}, " f"Part 2: {worker2_result.result}"
        return TaskResponse(task_id=message.task_id, result=combined_result)


runtime = SingleThreadedAgentRuntime()

await WorkerAgent.register(runtime, "worker", lambda: WorkerAgent("Worker Agent"))
await DelegatorAgent.register(runtime, "delegator", lambda: DelegatorAgent("Delegator Agent", "worker"))

runtime.start()

delegator = AgentId("delegator", "default")
result = await runtime.send_message(Task(task_id="main-task"), delegator)

print(f"Final result: {result.result}")
await runtime.stop_when_idle()

Delegator received task main-task.
worker/worker-1 starting task task-part-1
worker/worker-2 starting task task-part-2
worker/worker-1 finished task task-part-1
worker/worker-2 finished task task-part-2
Final result: Part 1: Results by worker/worker-1, Part 2: Results by worker/worker-2


Above we showed how sending a message from runtime to a specific agent (delegator) results in an expected response.

As an alternative to sending a message to `DelegatorAgent`, we can publish a message to the default topic. Because `DelegatorAgent` is subscribed to that topic, it will process the message. However, note that when publishing messages, any responses from the receiving agent are dropped.

In [127]:
runtime.start()

await runtime.publish_message(Task(task_id="main-task"), topic_id=DefaultTopicId())

await runtime.stop_when_idle()

Delegator received task main-task.
worker/worker-1 starting task task-part-1
worker/worker-2 starting task task-part-2
worker/worker-1 finished task task-part-1
worker/worker-2 finished task task-part-2
